In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# --- 1. Configuration ---
IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 32
DATA_DIR = '/content/drive/MyDrive/Model /dataset'  # The folder you created in Step 1

# --- 2. Load the Data ---
print("Loading and preparing dataset...")

# Load images from the directory
# 'label_mode' is 'binary' because we have only two classes.
# 'validation_split' reserves 20% of the data for testing the model's accuracy.
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# Get the class names (will be ['chest_xray', 'not_xray'])
class_names = train_ds.class_names
print(f"Classes found: {class_names}")

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- 3. Build the CNN Model ---
print("Building the model...")

model = keras.Sequential([
    # Rescale pixel values from [0, 255] to [0, 1]
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # First convolutional block
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Second convolutional block
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Third convolutional block
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    # Flatten the 3D features into a 1D vector
    layers.Flatten(),

    # A dense (fully-connected) layer
    layers.Dense(128, activation='relu'),

    # Output layer: 1 neuron with a 'sigmoid' activation.
    # Sigmoid is perfect for binary classification (outputs a value between 0 and 1)
    layers.Dense(1, activation='sigmoid')
])

# --- 4. Compile the Model ---
# We use 'binary_crossentropy' for a 2-class problem with a sigmoid output.
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Show a summary of the model
model.summary()

# --- 5. Train the Model ---
print("Training the model...")
EPOCHS = 10  # You can increase this if accuracy is low

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# --- 6. Save the Model ---
MODEL_NAME = 'chest_xray_detector.keras'
model.save(MODEL_NAME)
print(f"\nModel training complete! Saved as {MODEL_NAME}")

Loading and preparing dataset...
Found 5071 files belonging to 2 classes.
Using 4057 files for training.
Using 1014 files for validation.
Classes found: ['chest_xray', 'not_xray']
Building the model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,120,993 (8.09 MB)

 Trainable params: 2,120,993 (8.09 MB)

 Non-trainable params: 0 (0.00 B)

Training the model...
Epoch 1/10


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
import sys

# --- 1. Configuration ---
IMG_WIDTH = 128
IMG_HEIGHT = 128
MODEL_NAME = 'chest_xray_detector.keras'

# --- 2. Load the Model ---
try:
    model = tf.keras.models.load_model(MODEL_NAME)
    print(f"Model '{MODEL_NAME}' loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please make sure you have run 'train.py' first to create the model file.")
    sys.exit()

# --- 3. Create Prediction Function ---
def classify_image(image_path):
    """
    Loads an image, preprocesses it, and predicts if it's a chest X-ray.
    """
    try:
        # Load the image and resize it to the model's expected input size
        img = load_img(
            image_path,
            target_size=(IMG_HEIGHT, IMG_WIDTH)
        )

        # Convert the image to a NumPy array
        img_array = img_to_array(img)

        # Add an extra dimension to create a "batch" of 1
        # The model expects input shape (batch_size, height, width, channels)
        img_batch = np.expand_dims(img_array, axis=0)

        # Note: We don't need to rescale by 1./255 here because
        # the 'Rescaling' layer is already part of the saved model.

        # Make the prediction
        prediction = model.predict(img_batch, verbose=0)

        # The output is a value between 0 and 1 from the sigmoid function
        score = prediction[0][0]

        # We assume 'chest_xray' is class 0 and 'not_xray' is class 1
        # (This depends on the order tf.keras.utils.image_dataset_from_directory finds them)
        # Let's check the score. A score < 0.5 means class 0 (chest_xray)

        if score < 0.5:
            # Note: We use (1 - score) because accuracy is usually presented
            # as confidence in the *predicted* class.
            print(f"This is a: CHEST X-RAY (Confidence: {100 * (1 - score):.2f}%)")
        else:
            print(f"This is a: NOT A CHEST X-RAY (Confidence: {100 * score:.2f}%)")

    except FileNotFoundError:
        print(f"Error: The file '{image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- 4. Use the Function ---
# Get the image path from the command line argument
if len(sys.argv) < 2:
    print("How to use: python predict.py <path_to_your_image.jpg>")
    # Example:
    # python predict.py my_xray_test.jpg
    # python predict.py my_cat_photo.jpg
else:
    image_to_test = "/content/drive/MyDrive/Model /dataset/chest_xray/person88_virus_167.jpeg"
    classify_image(image_to_test)

Model 'chest_xray_detector.keras' loaded successfully.
This is a: CHEST X-RAY (Confidence: 99.99%)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os
import time
from google.colab import drive

# --- 1. Mount Google Drive ---
print("--- Step 1: Mounting Google Drive ---")
drive.mount('/content/drive')


# --- 2. Define Paths ---
# SLOW path on your Google Drive
DRIVE_SOURCE_DIR = '/content/drive/MyDrive/Model /dataset'

# FAST path on the local Colab disk
LOCAL_DATA_DIR = '/content/dataset'

# Where to save the final, trained model
MODEL_SAVE_PATH = '/content/drive/MyDrive/Model /chest_xray_detector_mobilenet.keras'


# --- 3. Copy Data from Drive to Local Disk (The Speed Fix) ---
print(f"\n--- Step 2: Copying dataset from {DRIVE_SOURCE_DIR} ---")
print("This may take a few minutes, but you only do it once.")
start_time = time.time()

# Create the local destination folder
os.makedirs(LOCAL_DATA_DIR, exist_ok=True)

# Use the 'cp' command to copy. It's much faster than Python's 'shutil'
# -r = recursive (all subfolders)
# -n = no-clobber (don't re-copy files that already exist)
!cp -r -n "{DRIVE_SOURCE_DIR}"/* "{LOCAL_DATA_DIR}"/

print(f"Copying complete! Took {time.time() - start_time:.2f} seconds.")


# --- 4. Configure GPU ---
print("\n--- Step 3: Configuring GPU ---")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"Found {len(gpus)} GPUs.")
    try:
        # Enable 'memory growth'
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("WARNING: No GPU found. Training will use the CPU.")


# --- 5. Set Model Configuration ---
print("\n--- Step 4: Setting Configuration ---")
IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 256  # <-- INCREASED BATCH SIZE to feed the GPU
EPOCHS = 10
print(f"Batch size set to: {BATCH_SIZE}")


# --- 6. Load Data (from the FAST local path) ---
print(f"\n--- Step 5: Loading data from {LOCAL_DATA_DIR} ---")
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    LOCAL_DATA_DIR,  # <-- USING FAST LOCAL PATH
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

class_names = train_ds.class_names
print(f"Classes found: {class_names}")


# --- 7. Preprocess Data ---
print("\n--- Step 6: Preprocessing data for MobileNetV2 ---")
def preprocess_data(image, label):
    return preprocess_input(image), label

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
print("Dataset preprocessing and caching complete.")


# --- 8. Build the Model with Transfer Learning ---
print("\n--- Step 7: Building MobileNetV2 model ---")
base_model = MobileNetV2(
    input_shape=(IMG_WIDTH, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model
base_model.trainable = False

# Add our custom head
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])


# --- 9. Compile the Model ---
print("\n--- Step 8: Compiling model ---")
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


# --- 10. Train the Model ---
print("\n--- Step 9: Starting model training (This should be fast now) ---")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


# --- 11. Save the Final Model to Google Drive ---
print(f"\n--- Step 10: Saving trained model to {MODEL_SAVE_PATH} ---")
model.save(MODEL_SAVE_PATH)
print(f"Model training complete! Saved to {MODEL_SAVE_PATH}")

--- Step 1: Mounting Google Drive ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Step 2: Copying dataset from /content/drive/MyDrive/Model /dataset ---
This may take a few minutes, but you only do it once.
Copying complete! Took 1.11 seconds.

--- Step 3: Configuring GPU ---
Found 1 GPUs.
Physical devices cannot be modified after being initialized

--- Step 4: Setting Configuration ---
Batch size set to: 256

--- Step 5: Loading data from /content/dataset ---
Found 5071 files belonging to 2 classes.
Using 4057 files for training.
Using 1014 files for validation.
Classes found: ['chest_xray', 'not_xray']

--- Step 6: Preprocessing data for MobileNetV2 ---
Dataset preprocessing and caching complete.

--- Step 7: Building MobileNetV2 model ---

--- Step 8: Compiling model ---


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


--- Step 9: Starting model training (This should be fast now) ---
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 963ms/step - accuracy: 0.8967 - loss: 0.2575 - val_accuracy: 0.9744 - val_loss: 0.1114
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.9778 - loss: 0.0815 - val_accuracy: 0.9842 - val_loss: 0.0339
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.9896 - loss: 0.0271 - val_accuracy: 0.9980 - val_loss: 0.0123
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.9994 - loss: 0.0122 - val_accuracy: 1.0000 - val_loss: 0.0083
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 1.0000 - loss: 0.0078 - val_accuracy: 1.0000 - val_loss: 0.0065
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 1.0000 - val_loss: 0.0055
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 1.0000 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 0.0047
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 7

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from google.colab import drive
import os

# --- 1. Mount Drive and Load Model ---
drive.mount('/content/drive')

MODEL_PATH = '/content/drive/MyDrive/Model/chest_xray_detector_mobilenet.keras'
IMAGE_TO_TEST = '/content/dataset/not_xray/some_test_image.jpg' # Change this path

# Load the model
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Make sure training is complete and the path is correct.")

# --- 2. Prediction Function ---
def classify_image(image_path, model):
    if not os.path.exists(image_path):
        print(f"Error: Image not found at {image_path}")
        # Try checking your Google Drive path if you didn't copy
        drive_image_path = f"/content/drive/MyDrive/Model /dataset/not_xray/{os.path.basename(image_path)}"
        if os.path.exists(drive_image_path):
            print(f"Found it in Drive, using: {drive_image_path}")
            image_path = drive_image_path
        else:
            print(f"Also not found in Drive: {drive_image_path}")
            return

    try:
        img = load_img(image_path, target_size=(128, 128))
        img_array = img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_batch)

        prediction = model.predict(img_preprocessed, verbose=0)
        score = prediction[0][0]

        if score < 0.5:
            confidence = 100 * (1 - score)
            print(f"Prediction: This is a CHEST X-RAY (Confidence: {confidence:.2f}%)")
        else:
            confidence = 100 * score
            print(f"Prediction: This is NOT A CHEST X-RAY (Confidence: {confidence:.2f}%)")

    except Exception as e:
        print(f"An error occurred during prediction: {e}")

# --- 3. Test the function ---
# Make sure to change IMAGE_TO_TEST to a real image path
# You can upload one to Colab or use a path from your copied dataset
classify_image(IMAGE_TO_TEST, model)